In [ ]:
## functions:
#calculate the workload of nodes  the betweeness of a node and set the weight to that
#calculate the betweeness of nodes 
#delete the node with the max betweeness or random
#keep number of connected componets and the number of nodes in the biggest component in list so that we can plot it
#distribute the weights of the deleted node to its neighbours
#repeat steps until the number of nodes in biggest connected component is below 50% or some other criteria



In [ ]:
pagerank = nx.pagerank(G5)
max_node = max(pagerank, key = pagerank.get)
# key=pagerank.get is used as a custom sorting key for the max function. 
# It tells Python to use the values from the pagerank dictionary 
# (the PageRank scores) to determine the maximum value.

print(f'The most central node is {max_node}, with a pagerank of {round(pagerank[max_node], 5)}')